# 量子测量噪声缓释——局域噪声模型

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

## 内容概要

串扰噪声是测量过程中一类复杂的噪声来源，对串扰噪声的高效的刻画方法对提高测量结果的缓释精度有很重要的意义。本教程将介绍一种可以高效刻画并缓释串扰噪声的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。

## 理论
我们已经介绍过测量噪声缓释可以分为两个过程，第一步是校准，通过制备校准电路，构建校准矩阵，从而获得噪声的信息，这一步可以通过**无结构校准**和**张量积校准**两种方式进行；第二步是缓释，通过获取的校准矩阵，我们可以对测量结果进行噪声缓释，这一步有许多方法，例如**矩阵逆方法** [1]，**最小二乘法**，**迭代贝叶斯方法**（Iterative Bayesian Unfolding，IBU）[2]，**Neumann 方法** [3]等。

### 局域噪声校准

上述的**无结构校准**很耗费资源，而**张量积校准**又要求不存在串扰（这在实验上并不是容易满足的条件）。IBM 提出一种基于连续时间马尔科夫过程（Continuous Time Markov Process，CTMP）[4] 来对测量噪声进行建模的方法，这种方法相比**张量积校准**可以很好地刻画串扰噪声，相比**无结构校准**又能节省很多资源。CTMP 方法认为，校准矩阵 $R$ 可以看作是微分方程

$$
\begin{align}
\dot{R}(t) = GR(t)
\end{align}
$$

的解，该微分方程的初始值是 $R(0)=I$。此时 $G$ 是一个连续时间马尔科夫过程的生成子，$\langle y\vert G \vert x\rangle$ 就表示从态 $x$ 转换到态 $y$ 且 $y\neq x$ 的概率。解微分方程我们可以得到校准矩阵的表达式

$$
\begin{align}
R = e^G.
\end{align}
$$

这也就意味着校准矩阵 $R$ 由生成矩阵 $G$ 唯一确定，而 $G$ 的表达式为

$$
\begin{align}
G = \sum_{i}λ_i G_i,
\end{align}
$$

其中 $λ_i$ 表示为由于错误概率，由实验上的校准数据获得，$G_i$ 是用于描述该错误所对应的生成器，是可以直接理论计算得到的值。

由于 $λ_i$ 刻画了串扰噪声，在讨论如何确定 $λ_i$ 之前，我们需要先确定串扰噪声的定义。串扰噪声与比特数有关，当给定的 $k$ 量子比特制备输入基态并进行测量，得到的输出态与输入态完全相反（即 $k$ 个量子比特全部被翻转），那么我们说这 $k$ 量子比特之间存在 $k$ 阶串扰噪声。直观上而言，$k$ 阶串扰噪声意味着这 $k$ 个量子比特是全关联的。假设我们要刻画 $S=\left\{Q_0,⋯,Q_{k-1} \right\}$ 这 $k$ 个量子比特的 $k$ 阶串扰噪声，我们先基于无结构模型从校准数据集计算得到 $2^k×2^k$ 的校准矩阵 $R^S$，那么该矩阵的 $2^k$ 个反向对角元素刻画了 $k$ 比特输出态与输入态完全相反的概率，即 $k$ 个量子比特全部翻转的概率。我们对 $R^S$ 求矩阵对数 ，所得矩阵的 $2^k$ 个反向对角元素定义为 $S$ 的 $k$ 阶串扰噪声权重系数 $\left\{λ_i \right\}_{i=0}^{2^k-1}$，它们完全刻画 $S$ 所表示的 $k$ 个量子比特存在的 $k$ 阶串扰噪声：

$$
\begin{align}
\left\{λ_i \right\}_{i=0}^{2^k-1}=antidiag \left(\log\left(R^S \right)\right),
\end{align}
$$

其中 $antidiag(R)$ 表示选取矩阵 $R$ 的所有反向对角元素（选取顺序为从左下角到右上角），而 $log(R)$ 表示矩阵对数运算。由权重系数的定义可知，$k$ 阶串扰噪声被 $2^k$ 个元素描述，每一个权重系数 $λ_i$ 刻画了一个具体的 $k$ 个量子比特翻转错误，该错误我们使用一个 $2^n×2^n$ 的生成器 $G_i$ 描述：

$$
\begin{align}
G_i  = |2^n-1 \oplus i⟩⟨i|-|i⟩⟨i|,
\end{align}
$$

其中 $\oplus$ 表示二进制异或操作 ，$2^n-1 \oplus i$ 表示将正整数 $i$ 二进制展开之后按位取反。直观上，$G_i$ 刻画了事件：“输入标准基量子态 $|i⟩$，输出要么是比特串 $i$，要么是完全翻转比特串 $2^n-1 \oplus i$”。这里需要强调的是，$G_i$ 默认张量到整个 $n$ 量子比特空间，因而是 $2^n×2^n$ 的矩阵。穷举 $i\in\left\{0,⋯,2^k-1\right\}$，我们一共得到 $2^k$ 个生成器。

实验表明，当两个量子比特之间不互联且相隔很远时，这两个比特之间的串扰很弱，于是我们可以利用这个性质，结合硬件本身的拓扑结构来选择量子测量设备中哪 $k$ 个量子比特之间需要考虑 $k$ 阶串扰噪声。一般而言，含有 $n$ 个量子比特的硬件拓扑结构可使用简单无向图 $Ω=(V,E)$ 描述，其中 $V$ 是节点集合（对应硬件中的量子比特），而 $E$ 是边集合（刻画硬件中量子比特的互联性）。我们假设硬件中某 $k$ 个量子比特之间存在 $k$ 阶串扰噪声，当且仅当该 $k$ 个量子比特对应的节点集合是简单无向图中的 $k$ 阶连通子图。直观上而言，如果这 $k$ 个量子比特对应的节点在无向图上是互相连通的，那么我们认为它们之间会存在 $k$ 阶串扰噪声。

事实上，我们不难发现，**无结构模型和张量积模型是局域噪声模型的两种特殊情况**，当我们只考虑 $k=1$ 阶串扰噪声时，就是张量积模型，当我们考虑 $k=n$ 阶串扰噪声时，就是无结构模型。
### 局域噪声修正

当我们使用**局域噪声校准**方法，获得生成矩阵 $G$ 后，我们可以将校准矩阵的逆 $R^{-1}$ 进行展开，从而对测量结果进行噪声缓释。令

$$
\begin{equation}
    \gamma = \max_{x\in\left\{0, 1\right\}^n}{-\langle x\vert G\vert x \rangle}.
\end{equation}
$$

定义矩阵

$$
\begin{equation}
    B = I + \gamma^{-1}G,
\end{equation}
$$

可以得到

$$
\begin{equation}
    R^{-1} = e^{-G} = e^{\gamma} \cdot e^{-\gamma B} = \sum_{\alpha=0}^{\infty}\frac{e^{\gamma}\left(-\gamma\right)^\alpha}{\alpha!}B^{\alpha}.
\end{equation}
$$

而 $\alpha$ 的取值可以通过两次迭代的距离小于某个阈值来截断。

## 实践

### 计算任务描述

下面我们通过一个 GHZ 态来进行测量噪声缓释的展示。

我们首先生成大量 $2$ 量子比特的 GHZ 态（严格来讲应该叫 Bell 态，为了描述方便，教程中统一为 GHZ 态），在 $Z$ 基下进行测量，得到输出态的统计信息，然后构造观测算符

$$
\begin{equation}
O = {\vert0\rangle\langle0\vert}^{\otimes2}+{\vert1\rangle\langle1\vert}^{\otimes2},
\end{equation}
$$

求它的期望值，理想情况下，结果应该是 1，然而如果有噪声影响，期望值将会不等于 $1$。

### 含噪结果

首先我们导入必要的库

In [ ]:
import copy
import numpy as np
import functools
import networkx as nx
import matplotlib.pyplot as plt
import seaborn
import pandas
from QCompute import *

from qcompute_qep.measurement.correction import CTMPCorrector
from qcompute_qep.measurement.utils import plot_histograms, dict2vector
from qcompute_qep.utils import expval_from_counts
from qcompute_qep.utils.types import get_qc_name
from qcompute_qep.utils.circuit import execute

然后我们需要从量易伏中获取 Token，来使用量子计算真机

In [ ]:
# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
Define.hubToken = "TOKEN"

# Set the default maximal number of measurement shots
MAX_SHOTS = 8192

我们开始构造 GHZ 态对应的量子电路，并通过 $Z$ 方向上的测量来获取输出态的统计结果

In [ ]:
qp = QEnv()
n = 2
qp.Q.createList(n)
H(qp.Q[0])
for i in range(1, n):
    CX(qp.Q[0], qp.Q[i])

MeasureZ(*qp.Q.toListPair())

然后我们构造观测量算符 $O$

In [ ]:
proj0 = np.array([[1, 0], [0, 0]]).astype(complex)
proj1 = np.array([[0, 0], [0, 1]]).astype(complex)
O = functools.reduce(np.kron, [proj0] * n) + functools.reduce(np.kron, [proj1] * n)

选择用来计算的量子计算机，其中 LocalBaiduSim2 是理想模拟器，而 CloudBaiduQPUQian 是百度量子计算研究所提供的量子真机

In [ ]:
ideal_qc = BackendName.LocalBaiduSim2

noisy_qc = BackendName.CloudBaiduQPUQian

noisy_qc_name = get_qc_name(noisy_qc)

获取量子计算机的执行结果（即输出态的统计信息），利用统计信息计算观测算符 $O$ 的期望值

In [ ]:
# WARNING! We must deep copy the quantum program since `execute` will destroy the quantum program

# Ideal case.
counts_ideal = execute(qp=copy.deepcopy(qp), qc=ideal_qc, shots=MAX_SHOTS)
val_ideal = expval_from_counts(O, counts_ideal)

print("The ideal expectation value is: {}".format(val_ideal))

# Noisy case.
counts_noisy = execute(qp=copy.deepcopy(qp), qc=noisy_qc, shots=MAX_SHOTS)
val_noisy = expval_from_counts(O, counts_noisy)

print("The noisy expectation value is: {}".format(val_noisy))

### 修正结果

得到计算结果后，我们便可以对其中含噪的情况进行修正。如前所述，测量噪声缓释一共是两步走：校准+修正。值得一提的是，我们是对输出态的统计结果进行修正，再利用修正的统计结果进行期望值的计算。接下来对局域噪声模型的效果一一进行展示。我们需要强调，对于局域噪声修正，只能选取局域噪声校准。同时，我们也必须考虑到量子计算机的拓扑结构。为了方便用户的使用，我们内置了常见量子计算机的拓扑结构，若用户使用的量子计算机不在内置的拓扑结构中，需要用户使用 networkx 自行设置拓扑结构。在本演示中，我们选择的量子计算机的拓扑结构已经内置，此处的 $k$ 表示串扰噪声阶数，默认为 $2$。

In [ ]:
corr_ctmp = CTMPCorrector(qc=noisy_qc, calibrator='ctmp', qubits=range(n), k=2)

# If the topology of the quantum computer is not built-in, users should set the topology by themselves.
# G = nx.Graph()
# G.add_nodes_from([0, 1, 2, 3, 4])
# G.add_edges_from([(0, 1), (1, 2), (1, 3), (3, 4)])
# corr_ctmp = CTMPCorrector(qc=noisy_qc, calibrator='ctmp', qubits=range(n), topo=G)

counts_ctmp = corr_ctmp.correct(counts_noisy)

# Compute the expectation value from corrected counts
val_ctmp = expval_from_counts(O, counts_ctmp)

print("The 'CTMP Calibrator + CTMP Corrector' "
      "mitigated expectation value is: {}".format(val_ctmp))

## 总结
在我们的演示中，我们讨论 $2$ 阶串扰噪声的情况，在局域噪声模型中，结合量子计算机的拓扑结构，我们只使用了 $xx$ 个校准电路，相比无结构校准模型节省资源的同时又保证了准确性，而相比张量积模型又能更精准刻画测量噪声，这显示了局域噪声模型的优越性。

## 参考资料

[1] Wikipedia contributors. "Moore–Penrose inverse." [Wikipedia](https://en.wikipedia.org/w/index.php?title=Moore%E2%80%93Penrose_inverse&oldid=1022461613), 2021.

[2] Nachman, Benjamin, et al. "Unfolding quantum computer readout noise." [npj Quantum Information](https://arxiv.org/abs/1910.01969) 6.1 (2020): 1-7.

[3] Wang, Kun, Yu-Ao Chen, and Xin Wang. "Measurement Error Mitigation via Truncated Neumann Series." [arXiv](https://arxiv.org/abs/2103.13856) preprint arXiv:2103.13856, 2021.

[4] Bravyi, Sergey, et al. "Mitigating measurement errors in multiqubit experiments." [Physical Review A](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.103.042605) 103.4 (2021): 042605.